In [1]:
from Naive_bayes import NaiveBayes
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )

y = [0 for x in range(0, 200)]
y += [1 for x in range(200, 400)]
y += [2 for x in range(400, 600)]
y += [3 for x in range(600, 800)]
y += [4 for x in range(800, 1000)]
y += [5 for x in range(1000, 1200)]
y += [6 for x in range(1200, 1400)]
y += [7 for x in range(1400, 1600)]
y += [8 for x in range(1600, 1800)]
y += [9 for x in range(1800, 2000)]

In [3]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
  
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   

In [4]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

In [5]:
X_train_mfeat, X_test_mfeat, y_train_mfeat, y_test_mefeat =  train_test_split(mfeat,y, test_size=0.10,random_state=42, shuffle=True, stratify=y )
X_train_mfeatFou, X_test_mfeatFou, y_train_mfeatFou, y_test_mefeatFou =  train_test_split(mfeatFou,y, test_size=0.10,random_state=42, shuffle=True,stratify=y)
X_train_mfeatKa, X_test_mfeatKa, y_train_mfeatKa, y_test_mefeatKa =  train_test_split(mfeatKa,y, test_size=0.10, random_state=42,shuffle=True,stratify=y)

In [6]:
X_trains = []


In [7]:
X_trains.append(X_train_mfeat)
X_trains.append(X_train_mfeatFou)
X_trains.append(X_train_mfeatKa)


In [8]:
nb = NaiveBayes()

In [9]:
nb.fit(X_train_views=X_trains, y_train_views=[y_train_mfeat,y_train_mfeatFou,y_train_mfeatKa])

In [10]:
y_pred,  v1 = nb.predict(X_tests=[X_test_mfeat,X_test_mfeatFou,X_test_mfeatKa]);

In [11]:
from sklearn.metrics import accuracy_score

In [22]:
acc_final= accuracy_score(y_test_mefeatKa, y_pred )

In [23]:
acc_final

0.94